In [ ]:
!pip install -r requirements.txt --user

In [ ]:
import time
import boto3

sm = boto3.Session().client("sagemaker")

In [ ]:
def get_regions():
    ec2 = boto3.client("ec2")
    regions = ec2.describe_regions()
    return [r["RegionName"] for r in regions["Regions"]]

In [ ]:
def search_projects(_project):
    projects = sm.list_projects(NameContains=_project)

    if "NextToken" in projects:
        next_token = projects["NextToken"]
        print(f"\n{next_token}")

    print("ProjectNames:")
    for project in projects["ProjectSummaryList"]:
        project_name = project["ProjectName"]
        print(f"\n{project_name}")

In [ ]:
def cleanup_experiment(_experiment_name):
    trials = sm.list_trials(ExperimentName=_experiment_name)["TrialSummaries"]
    print("TrialNames:")
    for trial in trials:
        trial_name = trial["TrialName"]
        print(f"\n{trial_name}")

        components_in_trial = sm.list_trial_components(TrialName=trial_name)
        print("\tTrialComponentNames:")
        for component in components_in_trial["TrialComponentSummaries"]:
            component_name = component["TrialComponentName"]
            print(f"\t{component_name}")
            sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
            try:
                # comment out to keep trial components
                sm.delete_trial_component(TrialComponentName=component_name)
            except:
                # component is associated with another trial
                continue
            # to prevent throttling
            time.sleep(0.5)
        sm.delete_trial(TrialName=trial_name)
    sm.delete_experiment(ExperimentName=_experiment_name)
    print(f"\nExperiment {_experiment_name} deleted")

In [ ]:
def cleanup_model(_model_name):
    models = sm.list_models(NameContains=_model_name)
    model_bias_job_definitions = sm.list_model_bias_job_definitions(NameContains=_model_name)
    model_explainability_job_definitions = sm.list_model_explainability_job_definitions(
        NameContains=_model_name
    )
    # model_metadata = sm.list_model_metadata(NameContains=_model_name)
    model_package_groups = sm.list_model_package_groups(NameContains=_model_name)
    model_quality_job_definitions = sm.list_model_quality_job_definitions(NameContains=_model_name)

    if "NextToken" in models:
        next_token = models["NextToken"]
        print(f"\n{next_token}")

    print("JobDefinitions:")
    for model in model_bias_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    print("JobDefinitions:")
    for model in model_explainability_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    # print("ModelNames:")
    # for model in model_metadata["Models"]:
    #     model_name = model["ModelName"]
    #     print(f"\n{model_name}")

    print("ModelPackageGroupNames:")
    for group in model_package_groups["ModelPackageGroupSummaryList"]:
        group_name = group["ModelPackageGroupName"]
        print(f"\n{group_name}")

        model_packages = sm.list_model_packages(ModelPackageGroupName=group_name)
        print("ModelPackageNames:")
        for package in model_packages["ModelPackageSummaryList"]:
            if "ModelPackageName" in package:
                package_name = package["ModelPackageName"]
                print(f"\n{package_name}")
                sm.delete_model_package(ModelPackageName=package_name)
        sm.delete_model_package_group(ModelPackageGroupName=group_name)

    print("JobDefinitions:")
    for model in model_quality_job_definitions["JobDefinitionSummaries"]:
        model_name = model["EndpointName"]
        print(f"\n{model_name}")

    print("ModelNames:")
    for model in models["Models"]:
        model_name = model["ModelName"]
        print(f"\n{model_name}")

In [ ]:
def cleanup_endpoint(_endpoint_name):
    endpoints = sm.list_endpoints(NameContains=_endpoint_name)

    if "NextToken" in endpoints:
        next_token = endpoints["NextToken"]
        print(f"\n{next_token}")

    print("EndpointNames:")
    for endpoint in endpoints["Endpoints"]:
        endpoint_name = endpoint["EndpointName"]
        sm.delete_endpoint(EndpointName=endpoint_name)
        print(f"\n{endpoint_name} deleted")

    #     components_in_trial = sm.list_trial_components(TrialName=trial_name)
    #     print("\tTrialComponentNames:")
    #     for component in components_in_trial["TrialComponentSummaries"]:
    #         component_name = component["TrialComponentName"]
    #         print(f"\t{component_name}")
    #         sm.disassociate_trial_component(TrialComponentName=component_name, TrialName=trial_name)
    #         try:
    #             # comment out to keep trial components
    #             sm.delete_trial_component(TrialComponentName=component_name)
    #         except:
    #             # component is associated with another trial
    #             continue
    #         # to prevent throttling
    #         time.sleep(0.5)
    #     sm.delete_trial(TrialName=trial_name)
    # sm.delete_experiment(ExperimentName=_experiment_name)
    # print(f"\nExperiment {_experiment_name} deleted")

In [ ]:
def cleanup_monitoring_schedules(_monitoring_schedule_name):
    sm.delete_monitoring_schedule(MonitoringScheduleName=_monitoring_schedule_name)

In [ ]:
def cleanup_endpoints(_endpoint_name):
    sm.delete_endpoint(EndpointName=_endpoint_name)

In [ ]:
def cleanup_project(project_name):
    sm.delete_project(ProjectName=project_name)

In [ ]:
experiment_name = "jumpstart-example-py-220927-0613"
cleanup_experiment(experiment_name)

monitoring_schedules = sm.list_monitoring_schedules()
for monitoring_schedule in monitoring_schedules["MonitoringScheduleSummaries"]:
    print(monitoring_schedule["MonitoringScheduleName"])
    monitoring_schedule_name = monitoring_schedule["MonitoringScheduleName"]
    cleanup_monitoring_schedules(monitoring_schedule_name)

endpoints = sm.list_endpoints()
for endpoint in endpoints["Endpoints"]:
    print(endpoint["EndpointName"])
    endpoint_name = endpoint["EndpointName"]
    cleanup_endpoints(endpoint_name)

In [ ]:
# get_regions()

In [ ]:
search_projects("animal")

In [ ]:
cleanup_model("AbaloneModel")

In [ ]:
cleanup_model("animal-classifier")

In [ ]:
cleanup_project("animal-classifier")

In [ ]:
cleanup_endpoint("sagemaker-sklearn-automl")